In [7]:
import torch
import pickle
import random
import torchvision
import numpy as np
import datetime as dt
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.training.supervised import GSS_greedy
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR100
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, make_classification_dataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy, StreamAccuracy

In [8]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [9]:
train_set = CIFAR100('Dataset/CIFAR100', train=True, download=True)
test_set = CIFAR100('Dataset/CIFAR100', train=False, download=True)

transforms_group = dict(
        eval=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                ]
            ),
            None,
        ),
        train=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                ]
            ),
            None,
        ),
    )

train_set = CIFAR100('Dataset/CIFAR100', train=True,download=True,)
test_set = CIFAR100('Dataset/CIFAR100', train=False,download=True,)
train_set = make_classification_dataset(train_set, transform_groups=transforms_group, initial_transform_group="train",)
test_set = make_classification_dataset(test_set, transform_groups=transforms_group, initial_transform_group="eval",)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 100
incremental = 10

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-2)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\AI\anaconda3\envs\lab\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\AI\anaconda3\envs\lab\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [11]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_GSS-Greedy_cifar100_avalanche_" + date)
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    StreamAccuracy(),
    loggers=[interactive_logger, tensor_logger])

## GSS-Greedy

In [12]:
train_batch = 512
eval_batch = 512
epoch = 50

strategies = GSS_greedy(model, optimizer, criterion, mem_size=2000, mem_strength=10, input_size=[3, 32, 32], train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [13]:
print("Starting experiment...")
results = []

for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|██████████| 10/10 [00:52<00:00,  5.23s/it]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.1452
100%|██████████| 10/10 [00:46<00:00,  4.60s/it]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3108
100%|██████████| 10/10 [00:46<00:00,  4.60s/it]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.3982
100%|██████████| 10/10 [00:46<00:00,  4.63s/it]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4314
100%|██████████| 10/10 [00:45<00:00,  4.57s/it]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4086
100%|██████████| 10/10 [00:46<00:00,  4.64s/it]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4664
100%|██████████| 10/10 [00:46<00:00,  4.63s/it]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4782
100%|██████████| 10/10 [00:46<00:00,  4.65s/it]
Epoch 7 ended.

KeyboardInterrupt: 